In [1]:
# Adjust atomic term energies (SOC matrix diagonals) to fit exptl energy levels
#    to obtain semiempirical term energies
# ** This will probably break if there are multiple terms with the same term symbol **
# ** If so, it can be fixed by adding ordinal prefixes to term symbols              **
# KKI 4/17/2023
import re, sys, glob, subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy, collections
sys.path.insert(0, '../karlib')
import chem_subs as chem
import molpro_subs as mpr

pd.set_option('display.max_rows', None)

In [2]:
# Excel spreadsheet of experimental levels from https://physics.nist.gov/PhysRefData/ASD/levels_form.html
#   Download as CSV; paste into a column in Excel; use Data -> Text to Columns -> Delimited -> Comma
#   Rename that worksheet with a name like "Fe" or "Fe+"
# Note that experimental levels might not be listed by increasing energy
xl_expt = 'exptl_levels.xlsx'
xl = pd.ExcelFile(xl_expt)

### Select atom and parity of interest

In [3]:
atom = 'Fe'  # a name like "Fe" or "Fe+"
parity = 'even'  #  choose 'even' or 'odd' or 'both'

### Select energy maximum for experimental terms

In [4]:
# In case of errors, try making this larger or smaller to match the theoretical calculation
termcut = 18000  # discard terms that lack levels below this energy (cm-1)

In [5]:
Ecol = 'Level (cm-1)'  # the exptl energy column
# display formatting
fmt = {'Eshift': '{:.1f}', Ecol: '{:.3f}', 'Pct': '{:.3f}', 'degen': '{:.0f}'}
for col in ['J', 'Ecalc', 'E_dif', 'Erel', 'Eshift', 'err', 'Eterm', 'cm-1', 'fitted']:
    fmt[col] =  fmt['Eshift']
for col in ['dif', 'Theory', 'ecm', 'SOC', 'RMSE']:
    fmt[col] = '{:.2f}'

In [6]:
if atom not in xl.sheet_names:
    print(f'No experimental data sheet for {atom}!')
else:
    dfexpt = pd.read_excel(xl, atom)
    # Delete any ionization limit
    dfexpt = dfexpt[dfexpt.Term != 'Limit']
    print(f'{len(dfexpt)} experimental levels for {atom} read from "{xl_expt}"')
    # Select by parity
    if parity == 'even':
        # discard odd levels ('Term' field ends with '*')
        dfexpt = dfexpt[~dfexpt.Term.str.contains('\*$')]
    elif parity == 'odd':
        dfexpt = dfexpt[dfexpt.Term.str.contains('\*$')]
    print(f'{len(dfexpt)} levels are of parity "{parity}"')
    # Select terms by energy
    lowTerms = []
    for term, grp in dfexpt.groupby('Term'):
        if (grp[Ecol] < termcut).any():
            lowTerms.append(term)
    print(f'There are {len(lowTerms)} assigned terms with levels below {termcut} cm-1:')
    print('   ', lowTerms)
    dfexpt = dfexpt[dfexpt.Term.isin(lowTerms)]
    nlevx = len(dfexpt)
    # parse 'Term' column to get simplified term labels
    def simplify(term):
        # extract the basic LS part of a decorated term label
        regex = re.compile('\d[SPDF-Z]')
        m = regex.search(term)
        if m:
            return m.group(0)
        else:
            # failed
            return '?'
    dfexpt['Tlbl'] = dfexpt.Term.apply(simplify)
    # Convert experimental 'J' and 'Level' to floats
    for col in ['J', Ecol]:
        dfexpt[col] = dfexpt[col].astype(float)
    # add degeneracy = 2J+1
    dfexpt['degen'] = 2 * dfexpt.J + 1
    # sort by energy (just in case)
    dfexpt = dfexpt.sort_values('Level (cm-1)')
    nmagx = int(dfexpt.degen.sum())
    print(f'There are {nlevx} levels of interest ({nmagx} magnetic sublevels)')
    display(dfexpt.style.format(fmt))
    gexpt = list(dfexpt.degen.astype(int))

846 experimental levels for Fe read from "exptl_levels.xlsx"
368 levels are of parity "even"
There are 4 assigned terms with levels below 18000 cm-1:
    ['a 3F', 'a 5D', 'a 5F', 'a 5P']
There are 16 levels of interest (96 magnetic sublevels)


,Configuration,Term,J,Prefix,Level (cm-1),Suffix,Uncertainty (cm-1),Lande,Leading percentages,Reference,Tlbl,degen
0,3d6.4s2,a 5D,4.0,nan,0.000,nan,nan,1.50020,100,L11631,5D,9
1,3d6.4s2,a 5D,3.0,nan,415.933,nan,0.001000,1.50034,100,nan,5D,7
2,3d6.4s2,a 5D,2.0,nan,704.007,nan,0.001000,1.50041,100,nan,5D,5
3,3d6.4s2,a 5D,1.0,nan,888.132,nan,0.001000,1.50022,100,nan,5D,3
4,3d6.4s2,a 5D,0.0,nan,978.074,nan,0.001000,nan,100,nan,5D,1
5,3d7.(4F).4s,a 5F,5.0,nan,6928.268,nan,0.001000,1.40021,100,nan,5F,11
6,3d7.(4F).4s,a 5F,4.0,nan,7376.764,nan,0.001000,1.35004,100,nan,5F,9
7,3d7.(4F).4s,a 5F,3.0,nan,7728.060,nan,0.001000,1.24988,100,nan,5F,7
8,3d7.(4F).4s,a 5F,2.0,nan,7985.785,nan,0.001000,0.99953,100,nan,5F,5
9,3d7.(4F).4s,a 5F,1.0,nan,8154.714,nan,0.001000,-0.014,100,nan,5F,3


### Take assignments at face value, i.e., apply eq. (1)

In [7]:
# No theoretical calculations are needed to use eq. (1)
xterms = []  # list of term labels
eterms = []  # list of term energies
tlbls = []  # simplified term labels
for Term, grp in dfexpt.groupby(['Term']):
    xterms.append(Term)
    emean = np.dot(grp.degen, grp[Ecol]) / grp.degen.sum()
    eterms.append(emean)
    tlbls.append(grp.Tlbl.values[0])
dfeq1 = pd.DataFrame({'Term': xterms, 'Eterm': eterms, 'Tlbl': tlbls}).sort_values('Eterm').reset_index(drop=True)
dfeq1['Erel'] = dfeq1.Eterm - dfeq1.Eterm.min()
print('Term energies (cm-1) using eq. (1) and tabulated assignments')
display(dfeq1.style.format(fmt))
SOC1 = -1 * np.round(dfeq1.at[0, 'Eterm'], 3)
print(f'The corresponding spin-orbit stabilization energy is SOC1 = {SOC1:.2f} cm-1')
cols = ['Case', 'RMSE', 'SOC'] + list(dfeq1.Tlbl.values)
dfsummary = pd.DataFrame(columns=cols)
for t in tlbls:
    fmt[t] = '{:.1f}'
# situation without theoretical info
row = ['expt only', np.nan, SOC1, *list(dfeq1.Erel.values)]
dfsummary.loc[0] = row

Term energies (cm-1) using eq. (1) and tabulated assignments


,Term,Eterm,Tlbl,Erel
0,a 5D,403.0,5D,0.0
1,a 5F,7459.8,5F,7056.8
2,a 3F,12407.4,3F,12004.4
3,a 5P,17684.6,5P,17281.6


The corresponding spin-orbit stabilization energy is SOC1 = -402.96 cm-1


### Specify Molpro SO-CI output file

In [8]:
#fsoc = 'fe_15Q21T_ctzdk_x2c.pro'
#fsoc = 'fe_ci_15Q7T_c5zdk_x2c.pro'
fsoc = 'fe_15Q7T_ctzdk_x2c.pro'

print(f'Reading MOLPRO file "{fsoc}"')
compAtom = mpr.stoichiometry(fsoc)
charge = mpr.total_charge(fsoc)
print(f'The atom is {compAtom} with charge {charge}')
# check for consistency with the experimental data that were read
if charge > 0: 
    compAtom += '+'
elif charge < 0:
    compAtom += '-'
if abs(charge) > 1:
    compAtom += f'{abs(charge)}'
        
if compAtom != atom:
    print(f'*** exptl atom = {atom} is different')
PG = mpr.read_compgroup(fsoc)
print(f'The computational point group is {PG}')

Reading MOLPRO file "fe_15Q7T_ctzdk_x2c.pro"
The atom is Fe with charge 0
The computational point group is Ci


In [9]:
SOCI = mpr.fullmatSOCI(fsoc, hybrid=True)
vals_original = SOCI.vals.copy()
#vecs_original = SOCI.vec.copy()
matcopy = SOCI.matrix.copy()
#print('SO-CI matrix diagonal:')
#np.set_printoptions(suppress = True)
#print(np.round(matcopy.diagonal().real, 1))

Computational group = Ci
CASSCF states:
    15 Quintet
     7 Triplet
Replacing MRCI+Q energies by HLSDIAG values


In [10]:
dfterm = SOCI.average_terms(be_close=['Energy', 'Edav', 'Eref', 'dipZ', 'C0'])
print('Averaged terms:')
display(dfterm)

Averaged terms:


,Term,dipZ,Edav,idx,ecm
0,5D,0.0,-1272.167874,"[0, 1, 2, 3, 4]",0.0
1,5F,0.0,-1272.139424,"[5, 6, 7, 8, 10, 9, 11]",6244.1
2,3F,0.0,-1272.113972,"[17, 18, 19, 15, 16, 21, 20]",11830.1
3,5P,0.0,-1272.088406,"[12, 13, 14]",17441.3


In [11]:
# Create global 'term_order'
term_order = dfterm.Term.values
print('Term order: ', term_order)
term_energies = dfterm.ecm.values

Term order:  ['5D' '5F' '3F' '5P']


In [12]:
def levels_term_energies(term_order, term_energies, SOCI):
    # Return the SO-CI energies from using new term energies
    # 'term_order' is array of term symbols
    # 'term_energies' is array of corresponding energies (cm-1)
    # 'SOCI' is a fullmatSOCI() object
    #  Install the term energies along the SOCI.matrix diagonal and rediagonalize
    term_dict = dict(zip(term_order, term_energies))
    newdiag = SOCI.matrix.diagonal().copy()
    for ibs in range(len(newdiag)):
        j = SOCI.sob_ici[ibs]
        term = SOCI.mrci[j].Term
        # install the new energy for the term
        newdiag[ibs] = term_dict[term]
    # update the matrix
    np.fill_diagonal(SOCI.matrix, newdiag)
    SOCI.diagonalize(store=True, vectors=True, sortval=False)
    return SOCI.vals

In [13]:
def impose_degeneracy(vals, glist):
    # Given a list of energies and a list of degeneracies,
    #    return a list of lists of degenerate indices
    # Do not assume any ordering

    ndat = len(vals)
    gtot = np.sum(glist)
    if ndat != gtot:
        chem.print_err('', f'There are {ndat} energies but degeneracies sum to {gtot}')
    idxl = chem.find_degenerate(vals, tol=2) # First try the simple way
    g = [len(idx) for idx in idxl]
    target = collections.Counter(glist)  # counts for degeneracies
    #print('>>>target = ', target)
    if collections.Counter(g) == target:
        # this meets the requirements
        return idxl
    # Simple way did not work but is probably close
    # Rank groups by how isolated they are
    a = np.array(vals)
    # 'distmat' holds distances between all energies
    distmat = scipy.spatial.distance_matrix(a.reshape(-1,1), a.reshape(-1,1))
    def grp_dist():
        # distances between groups
        ngrp = len(idxl)
        gdist = np.zeros((ngrp, ngrp)) + np.inf
        for i, idx in enumerate(idxl):
            for j, jdx in enumerate(idxl):
                if j == i:
                    continue
                x = distmat[np.ix_(idx, jdx)]
                emin = np.min(x)
                gdist[i, j] = gdist[j, i] = emin
        return gdist
    while len(idxl) > len(glist):
        # join some groups 
        # 'gdist' is distances between all groups
        print('>', end='')
        gdist = grp_dist()
        # dmin[i] is the closest that group #i comes to any other group
        dmin = gdist.min(axis=0)
        imin = np.argmin(dmin)
        jmin = np.argmin(gdist[:, imin])
        # join groups imin and jmin
        imin, jmin = min(imin, jmin), max(imin, jmin)  # make imin < jmin
        lnew = idxl[imin] + idxl[jmin]
        idxl[imin] = lnew
        idxl.pop(jmin)
    # Are we done yet?
    g = [len(idx) for idx in idxl]
    if collections.Counter(g) == target:
        return idxl
    while len(idxl) < len(glist):
        # split some groups
        print('<', end='')
        print('len(idxl) = ', len(idxl))
        ibig = np.argmax(g)  # find the largest group
        if g[ibig] <= max(glist):
            # the biggest group is not too big
            # find the biggest group that occurs too often
            g = [len(idx) for idx in idxl]
            gcount = collections.Counter(g)  # counts for degeneracies
            # are there incorrect degeneracies?
            countdiff = gcount - target
            print('target =', target)
            print('gcount = ', gcount)
            print('countdiff =', countdiff)
            print('len(idxl) = ', len(idxl))
            if countdiff:
                gmax = max(countdiff.keys())
                print(f'split up {gmax}')
            # Find the group of size 'gmax' that has the largest internal gap
            ibigs = [i for i, degen in enumerate(g) if degen == gmax]
            print('ibigs =', ibigs)
            mingap = np.inf
            ibig = None
            for i in ibigs:
                x = min(distmat[np.ix_(idxl[i], idxl[i])])
                if x < mingap:
                    mingap = x
                    ibig = i
        # split group #ibig
        imat = distmat[idxl[ibig]]  # distances internal to the big group
        # find the largest near-diagonal
        neardiag = np.diagonal(imat, offset=1)
        i = np.argmax(neardiag)
        imax = idxl[ibig][i]
        jmax = idxl[ibig][i+1]
        # split between imax and jmax
        idxt = idxl.copy()
        idxl = []
        for i, idx in enumerate(idxt):
            if i == ibig:
                idxl.append(idxt[ibig][:i+1])
                idxl.append(idxt[ibig][i+1:])
            else:
                idxl.append(idx)
    # Are we done yet?
    g = [len(idx) for idx in idxl]
    if collections.Counter(g) == target:
        return idxl
    print(idxl, g, 'Right number of groups but wrong degeneracies')
    return
##
def average_imposed_degeneracy(vals, glist):
    # Given a list of energies and a list of degeneracies,
    # Return:  list of averages, list of list of indices into vals[]
    a = np.array(vals)
    idxl = impose_degeneracy(a, glist)
    avg = []
    for idx in idxl:
        avg.append(np.mean(a[idx]))
    return avg, idxl
##

In [14]:
def compute_rmse(dfexpt, vals, glist, DFret=False):
    # Given SO-CI level energies return their RMS error 
    # 'dfexpt' is the DataFrame of exptl level energies
    # 'vals' is the array/list of theoretical levels
    # 'glist' is the list of level degeneracies to impose
    # 'tol' is in cm-1; used to determine degeneracy and value of J
    dfcomp = dfexpt[['Term', 'J', 'Level (cm-1)', 'Tlbl', 'degen']].copy()
    try:
        elvl, idxl = average_imposed_degeneracy(vals, glist)
    except ValueError:
        display(dfcomp.style.format(fmt))
        np.set_printoptions(suppress = True)
        print(np.round(vals, 0))
    gl = [len(idx) for idx in idxl]  # degeneneracies
    dftheory = pd.DataFrame({'Ecalc': elvl, 'degen': gl})
    dftheory['Erel'] = dftheory.Ecalc - dftheory.Ecalc.min()
    # sort by energy (just in case)
    dftheory = dftheory.sort_values('Ecalc')
    #display(dftheory)
    theory = []
    used = []    # match each theor. level only to one exptl
    for iex, row in dfcomp.iterrows():
        gex = row.degen  # exptl degeneracy
        for jth, jrow in dftheory.iterrows():
            # both DFs are ordered by energy; assume that they roughly correspond
            if (jth not in used) and (jrow.degen == gex):
                # a match
                theory.append(jrow.Erel)
                used.append(jth)
                break
    
    # If number of levels does not agree between expt and theory,
    #   try changing the tol for detecting degeneracies

    dfcomp['Theory'] = theory
    dfcomp['dif'] = dfcomp.Theory - dfcomp['Level (cm-1)']
    #display(dfcomp.style.format(fmt))
    rmse = np.sqrt( (dfcomp.dif ** 2).mean() )
    if DFret:
        return rmse, dfcomp
    else:
        return rmse

In [15]:
def rmse_fun(term_energies):
    # Uses globals
    # Return RMSE given term energies
    # Do not allow the lowest term energy to change
    global dfexpt, SOCI, term_order, gexpt
    vals = levels_term_energies(term_order, term_energies, SOCI)
    rmse = compute_rmse(dfexpt, vals, gexpt)
    return rmse
def obj_fun(exc_terme):
    # Given only excited term energies (assuming ground=0)
    #   return the RMSE
    terme = [0] + list(exc_terme)
    rmse = rmse_fun(terme)
    return rmse

In [16]:
# If this gives error about (# energies) != (degeneracies sum), try adjusting energy max "termcut" in cell #4
rmse_original, dforig = compute_rmse(dfexpt, vals_original, gexpt, DFret=True)  # as received from MOLPRO
SOC0 = vals_original[0]
print(f'Original RMSE = {rmse_original:.2f} cm-1 with SOC = {SOC0:.2f} cm-1')
display(dforig.style.format(fmt))

Original RMSE = 466.79 cm-1 with SOC = -397.12 cm-1


,Term,J,Level (cm-1),Tlbl,degen,Theory,dif
0,a 5D,4.0,0.000,5D,9,0.00,0.00
1,a 5D,3.0,415.933,5D,7,397.52,-18.41
2,a 5D,2.0,704.007,5D,5,695.59,-8.42
3,a 5D,1.0,888.132,5D,3,894.27,6.14
4,a 5D,0.0,978.074,5D,1,993.60,15.52
5,a 5F,5.0,6928.268,5F,11,6095.56,-832.71
6,a 5F,4.0,7376.764,5F,9,6543.36,-833.40
7,a 5F,3.0,7728.060,5F,7,6907.78,-820.28
8,a 5F,2.0,7985.785,5F,5,7184.26,-801.53
9,a 5F,1.0,8154.714,5F,3,7369.95,-784.76


In [17]:
# Use the averaged term energies--expect little change
rmse_avgd = rmse_fun(term_energies)
print(f'Using averaged input term energies, RMSE = {rmse_avgd:.2f} cm-1')
row = ['Before fit', rmse_avgd, SOC0] + list(dfterm.ecm.values)
dfsummary.loc[1] = row

Using averaged input term energies, RMSE = 466.80 cm-1


In [18]:
def freport(xvec):
    # callback function to monitor minimization
    freport.counter += 1
    print(f'{freport.counter:5d}', end='')
    return
freport.counter = 0

In [19]:
# Minimize the RMSE
exc_terme = list(term_energies)[1:]  # only excited terms; assume ground term = 0 energy
result = scipy.optimize.minimize(obj_fun, exc_terme, method='Nelder-Mead', callback=freport)

    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74   75   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96   97   98

In [20]:
print(f'After minimization, RMSE = {result.fun:.2f} cm-1')
terme = [0] + list(result.x)
Eterm = dict(zip(term_order, terme))
#print('Fitted term energies:')
#chem.print_dict(Eterm)
vals = levels_term_energies(term_order, terme, SOCI)
rmse, dfcomp = compute_rmse(dfexpt, vals, gexpt, DFret=True)
print('\nExptl vs fitted level energies:')
display(dfcomp.style.format(fmt))
SOCfit = vals[0]
print(f'The lowest level energy = SOCfit = {SOCfit:.2f} cm-1')

After minimization, RMSE = 20.19 cm-1

Exptl vs fitted level energies:


,Term,J,Level (cm-1),Tlbl,degen,Theory,dif
0,a 5D,4.0,0.000,5D,9,0.00,0.00
1,a 5D,3.0,415.933,5D,7,397.52,-18.42
2,a 5D,2.0,704.007,5D,5,695.58,-8.42
3,a 5D,1.0,888.132,5D,3,894.26,6.13
4,a 5D,0.0,978.074,5D,1,993.59,15.52
5,a 5F,5.0,6928.268,5F,11,6910.59,-17.68
6,a 5F,4.0,7376.764,5F,9,7357.35,-19.41
7,a 5F,3.0,7728.060,5F,7,7721.86,-6.20
8,a 5F,2.0,7985.785,5F,5,7998.81,13.02
9,a 5F,1.0,8154.714,5F,3,8184.97,30.26


The lowest level energy = SOCfit = -397.62 cm-1


In [21]:
row = ['After fit', rmse, SOCfit] + list(terme)
dfsummary.loc[2] = row

In [22]:
display(dfsummary.style.format(fmt).hide_index())
print(f'Input file: {fsoc}')

Case,RMSE,SOC,5D,5F,3F,5P
expt only,nan,-402.96,0.0,7056.8,12004.4,17281.6
Before fit,466.80,-397.12,0.0,6244.1,11830.1,17441.3
After fit,20.19,-397.62,0.0,7059.1,11998.1,17281.1


Input file: fe_15Q7T_ctzdk_x2c.pro
